## 02_nvflare 
#### <span style="color:red">Change to Image kernel</span>.

In [94]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [95]:
!git clone https://github.com/NVIDIA/NVFlare.git

fatal: destination path 'NVFlare' already exists and is not an empty directory.


In [96]:
%%bash
# Provision
server_hostname=$(hostname -s)
client_port=28005
admin_port=28003

# script
rm -rf secure_project workspace
cp NVFlare/examples/advanced/cifar10/cifar10-real-world/workspaces/secure_project.yml project.yml
sed -i "s/localhost/${server_hostname}/g" project.yml
sed -i "s/8102/${client_port}/g" project.yml
sed -i "s/8103/${admin_port}/g" project.yml
nvflare provision -p project.yml -w workspace
mv workspace/secure_project .
rm -rf workspace 
rm project.yml
mkdir -p logs

Project yaml file: /home/g00cjz00/demo2024/flag/project.yml.
Generated results can be found under /home/g00cjz00/demo2024/flag/workspace/secure_project/prod_00.  Builder's wip folder removed.


In [113]:
# Shutdown all job
!ps -ef |grep sub_start.sh  | awk '{print $2}' | xargs kill -9 
!ps -ef |grep nvflare.private.fed.app  | awk '{print $2}' | xargs kill -9 

kill: (186192): No such process
kill: 

In [104]:
%%bash 
# server
server_hostname=$(hostname -s)
client_port=28005
admin_port=28003
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
rm -f ${DIR}/secure_project/prod_00/${server_hostname}/daemon_pid.fl
nohup ${DIR}/secure_project/prod_00/${server_hostname}/startup/sub_start.sh > ./logs/${server_hostname}.log 2>&1 &

In [105]:
%%bash 
# site-1
client_hostname=site-1
client_port=28005
admin_port=28003
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
rm -f ${DIR}/secure_project/prod_00/${client_hostname}/daemon_pid.fl
nohup ${DIR}/secure_project/prod_00/${client_hostname}/startup/sub_start.sh > ./logs/${client_hostname}.log 2>&1 &

In [106]:
%%bash 
# site-2
client_hostname=site-2
client_port=28005
admin_port=28003
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
rm -f ${DIR}/secure_project/prod_00/${client_hostname}/daemon_pid.fl
nohup ${DIR}/secure_project/prod_00/${client_hostname}/startup/sub_start.sh > ./logs/${client_hostname}.log 2>&1 &

In [109]:
## admin
!rm -rf secure_project/prod_00/admin@nvidia.com/transfer/hello-pt
!cp -rf NVFlare/examples/hello-world/hello-pt/jobs/hello-pt secure_project/prod_00/admin@nvidia.com/transfer/hello-pt

In [111]:
# check client status 
import sys
cmd="check_status client"
from nvflare.fuel.flare_api.flare_api import new_secure_session
admin_dir="secure_project/prod_00/admin@nvidia.com"
sess = new_secure_session("admin@nvidia.com",admin_dir)
result=sess.api.do_command(cmd)
print(result)

{'time': '2024-02-16 00:10:49.374814', 'data': [{'type': 'table', 'rows': [['CLIENT', 'APP_NAME', 'JOB_ID', 'STATUS'], ['site-1', 'app', '7129d923-79a6-43cc-ac52-eb92e75063cc', 'started'], ['site-2', 'app', '7129d923-79a6-43cc-ac52-eb92e75063cc', 'started']]}], 'meta': {}, 'status': <APIStatus.SUCCESS: 'SUCCESS'>}


In [110]:
# submit job
import sys
cmd="submit_job hello-pt"
from nvflare.fuel.flare_api.flare_api import new_secure_session
admin_dir="secure_project/prod_00/admin@nvidia.com"
sess = new_secure_session("admin@nvidia.com",admin_dir)
result=sess.api.do_command(cmd)
print(result)

{'time': '2024-02-16 00:10:30.190764', 'data': [{'type': 'string', 'data': 'Submitted job: 7129d923-79a6-43cc-ac52-eb92e75063cc'}, {'type': 'success', 'data': ''}], 'meta': {'status': 'ok', 'info': '', 'job_id': '7129d923-79a6-43cc-ac52-eb92e75063cc'}, 'status': <APIStatus.SUCCESS: 'SUCCESS'>}


In [112]:
# list_jobs
import sys
cmd="list_jobs"
from nvflare.fuel.flare_api.flare_api import new_secure_session
admin_dir="secure_project/prod_00/admin@nvidia.com"
sess = new_secure_session("admin@nvidia.com",admin_dir)
result=sess.api.do_command(cmd)
print(result)

{'time': '2024-02-16 00:10:53.167537', 'data': [{'type': 'table', 'rows': [['JOB ID', 'NAME', 'STATUS', 'SUBMIT TIME', 'RUN DURATION'], ['d39e50f8-a0a5-4024-ab76-f55c59cecd55', 'hello-pt', 'FINISHED:COMPLETED', '2024-02-15T23:50:43.627923+08:00', '0:01:58.516343'], ['79251c20-1168-4ab6-aed4-8097a8cea1cb', 'hello-pt', 'FINISHED:COMPLETED', '2024-02-15T23:58:31.029155+08:00', '0:01:40.254268'], ['7129d923-79a6-43cc-ac52-eb92e75063cc', 'hello-pt', 'RUNNING', '2024-02-16T00:10:30.195891+08:00', '0:00:22.826286']]}, {'type': 'success', 'data': ''}], 'meta': {'jobs': [{'job_id': 'd39e50f8-a0a5-4024-ab76-f55c59cecd55', 'job_name': 'hello-pt', 'status': 'FINISHED:COMPLETED', 'submit_time': '2024-02-15T23:50:43.627923+08:00', 'duration': '0:01:58.516343'}, {'job_id': '79251c20-1168-4ab6-aed4-8097a8cea1cb', 'job_name': 'hello-pt', 'status': 'FINISHED:COMPLETED', 'submit_time': '2024-02-15T23:58:31.029155+08:00', 'duration': '0:01:40.254268'}, {'job_id': '7129d923-79a6-43cc-ac52-eb92e75063cc', 'j